# 🧪 Swin 모델 단위 테스트

이 노트북은 Swin 모델의 주요 기능을 테스트합니다:
- 모델 구조 및 파라미터 확인
- 샘플 추론 및 GPU 동작 확인

In [1]:
# [1] 프로젝트 루트 디렉토리로 이동
import os                                                   # OS 모듈 임포트
import sys                                                  # 시스템 모듈 임포트
os.chdir("../../../")                                       # 프로젝트 루트로 이동
print("현재 작업 디렉토리:", os.getcwd())                      # 현재 디렉토리 출력

현재 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN


In [2]:
# [2] 폰트 설정 및 경고 억제

# 경고 메시지 억제용 모듈 임포트
#!sudo apt -get install -y fonts-nanum  # 나눔폰트 설치 (필요시 주석 해제) 설치 후 vscode 재시작 필요
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 적용 및 시각화 환경 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔고딕 폰트 경로 및 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 폰트 등록 및 설정
fe = fm.FontEntry(fname=font_path, name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

# 글자 겹침 방지를 위한 레이아웃 설정
plt.rcParams['figure.autolayout'] = True
plt.rcParams['axes.titlepad'] = 20

In [8]:
# [3] 라이브러리 및 유틸리티 임포트
# 모델, 설정, 로거 등 필요한 모듈 임포트
import torch  # 파이토치 임포트
from src.models.build import build_model, get_recommended_model  # 모델 빌드 함수 및 추천 모델 함수 임포트
from src.utils.common import load_yaml  # 설정 로드 함수 임포트
from src.utils.unit_test_logger import create_test_logger  # 단위 테스트 로거 생성 함수 임포트
test_logger = create_test_logger("swin_model_test")  # 단위 테스트 로거 초기화

📝 단위 테스트 시작: swin_model_test
📝 로그 디렉토리: notebooks/modular/unit_tests/swin_model_test/20250905_234014


In [4]:
# [4] GPU 자동 체크
# GPU 사용 가능 여부 확인
if torch.cuda.is_available():  # GPU 사용 가능 여부 확인
    print(f'✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}')  # GPU 이름 출력
else:
    print('⚠️ GPU 사용 불가, CPU로 실행됩니다')  # CPU 사용 안내

✅ GPU 사용 가능: NVIDIA GeForce RTX 4090


In [10]:
# [5] 설정 파일 로드 및 모델 이름 확인
# 설정 파일에서 모델 정보 불러오기
try:
    cfg = load_yaml("configs/train_highperf.yaml")  # 설정 파일 로드
    
    # 추천 모델 이름 가져오기
    model_name = get_recommended_model(cfg['model']['name'])
    print(f'모델 이름: {cfg["model"]["name"]} -> {model_name}')
    print(f'클래스 수: {cfg["data"]["num_classes"]}')
    print(f'이미지 크기: {cfg["train"]["img_size"]}')
    print('✅ 설정 파일 로드 완료')
except Exception as e:
    print(f'⚠️ 설정 파일 로드 실패: {e}')

모델 이름: swin_base_384 -> swin_base_patch4_window12_384_in22k
클래스 수: 17
이미지 크기: 384
✅ 설정 파일 로드 완료


In [11]:
# [6] Swin 모델 생성
# 설정에 따른 Swin 모델 빌드
try:
    model = build_model(
        name=model_name,                        # 추천 모델 이름 사용
        num_classes=cfg['data']['num_classes'], # 클래스 수
        pretrained=cfg['model']['pretrained'],  # pretrained 가중치 사용
        drop_rate=cfg['model']['drop_rate'],    # dropout 비율
        drop_path_rate=cfg['model']['drop_path_rate']  # stochastic depth 비율
    )
    print(f'✅ 모델 생성 완료: {model_name}')
    print(f'모델 총 파라미터 수: {sum(p.numel() for p in model.parameters()):,}')
    
    # GPU로 모델 이동
    if torch.cuda.is_available():
        model = model.cuda()
        print('✅ 모델을 GPU로 이동 완료')
    else:
        print('ℹ️ CPU에서 모델 실행')
        
except Exception as e:
    print(f'⚠️ 모델 생성 실패: {e}')

✅ 모델 생성 완료: swin_base_patch4_window12_384_in22k
모델 총 파라미터 수: 86,896,009
✅ 모델을 GPU로 이동 완료


In [12]:
# [7] 샘플 추론 테스트
# 더미 입력으로 모델 추론 테스트
try:
    # 더미 입력 생성
    dummy_input = torch.randn(1, 3, cfg['train']['img_size'], cfg['train']['img_size'])
    print(f'더미 입력 shape: {dummy_input.shape}')
    
    # GPU 사용 시 입력도 GPU로 이동
    if torch.cuda.is_available():
        dummy_input = dummy_input.cuda()
        print('✅ 입력을 GPU로 이동 완료')
    
    # 샘플 추론 실행
    with torch.no_grad():  # 그래디언트 계산 비활성화
        output = model(dummy_input)
    
    print(f'✅ 샘플 추론 결과 shape: {output.shape}')
    print(f'출력 값 범위: [{output.min().item():.4f}, {output.max().item():.4f}]')
    
except Exception as e:
    print(f'⚠️ 샘플 추론 실패: {e}')

더미 입력 shape: torch.Size([1, 3, 384, 384])
✅ 입력을 GPU로 이동 완료
✅ 샘플 추론 결과 shape: torch.Size([1, 17])
출력 값 범위: [-0.1981, 0.4276]


In [13]:
# [8] 테스트 결과 저장 및 완료
# 모든 테스트 결과를 정리하여 저장
try:
    # 테스트 결과 딕셔너리 생성
    result = {
        'model_name': cfg['model']['name'],
        'actual_model_name': model_name,
        'num_classes': cfg['data']['num_classes'],
        'input_shape': list(dummy_input.shape),
        'output_shape': list(output.shape),
        'total_params': sum(p.numel() for p in model.parameters()),
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'output_min': float(output.min().item()),
        'output_max': float(output.max().item())
    }
    
    # 테스트 결과 저장
    test_logger.save_test_result('swin_model_test', result)
    test_logger.finalize_test()
    
    print('✅ 모든 테스트 완료!')
    print(f'📁 결과 저장 위치: {test_logger.base_dir}')
    print('✅ Swin 모델 테스트 성공적으로 완료')
    
except Exception as e:
    print(f'⚠️ 결과 저장 실패: {e}')

📝 테스트 결과 저장: swin_model_test
✅ 테스트 완료! 총 소요 시간: 224.19초
✅ 결과 요약: notebooks/modular/unit_tests/swin_model_test/20250905_234014/test_summary.json

🏁 단위 테스트 완료: swin_model_test
📁 결과 디렉토리: notebooks/modular/unit_tests/swin_model_test/20250905_234014
⏱️ 소요 시간: 224.19초
📊 테스트 섹션 수: 1
✅ 모든 테스트 완료!
📁 결과 저장 위치: notebooks/modular/unit_tests/swin_model_test/20250905_234014
✅ Swin 모델 테스트 성공적으로 완료


In [14]:
# [9] 저장된 테스트 결과 확인
# test_summary.json 파일 내용 출력 및 디렉토리 구조 확인
try:
    from pathlib import Path
    import json
    
    # test_summary.json 파일 경로
    summary_path = Path(test_logger.base_dir) / 'test_summary.json'
    
    print("📁 결과 저장 디렉토리:")
    print(f"   {test_logger.base_dir}")
    print()
    
    # 저장된 파일들 확인
    if Path(test_logger.base_dir).exists():
        print("📄 저장된 파일 목록:")
        for item in Path(test_logger.base_dir).rglob("*"):
            if item.is_file():
                relative_path = item.relative_to(test_logger.base_dir)
                file_size = item.stat().st_size
                print(f"   📄 {relative_path} ({file_size:,} bytes)")
        print()
    
    # test_summary.json 내용 출력
    if summary_path.exists():
        print("📋 테스트 요약 내용:")
        with open(summary_path, 'r', encoding='utf-8') as f:
            summary_content = f.read()
        print(summary_content)
    else:
        print("⚠️ test_summary.json 파일을 찾을 수 없습니다.")
        
except Exception as e:
    print(f'⚠️ 저장된 결과 확인 실패: {e}')

📁 결과 저장 디렉토리:
   notebooks/modular/unit_tests/swin_model_test/20250905_234014

📄 저장된 파일 목록:
   📄 test_summary.json (802 bytes)
   📄 results/swin_model_test_result.json (412 bytes)
   📄 logs/swin_model_test_20250905_234014.log (1,010 bytes)

📋 테스트 요약 내용:
{
  "test_name": "swin_model_test",
  "start_time": "2025-09-05T23:40:14.002609",
  "end_time": "2025-09-05T23:43:58.195535",
  "duration_seconds": 224.192926,
  "total_sections": 1,
  "results": {
    "swin_model_test": {
      "model_name": "swin_base_384",
      "actual_model_name": "swin_base_patch4_window12_384_in22k",
      "num_classes": 17,
      "input_shape": [
        1,
        3,
        384,
        384
      ],
      "output_shape": [
        1,
        17
      ],
      "total_params": 86896009,
      "device": "cuda",
      "output_min": -0.19808129966259003,
      "output_max": 0.42759647965431213,
      "timestamp": "2025-09-05T23:43:58.194530",
      "section": "swin_model_test"
    }
  },
  "log_directory": "noteboo

## 🏆 Swin 모델 단위 테스트 결과 요약

### 📋 **테스트 실행 단계**
1. **설정 파일 로드**: `configs/train_highperf.yaml`에서 모델 및 학습 설정 불러오기
2. **모델 생성**: Swin Transformer 모델 빌드 및 GPU 이동
3. **샘플 추론**: 더미 입력으로 추론 테스트 실행
4. **결과 저장**: 모든 테스트 결과를 체계적으로 저장

### 📊 **테스트 결과 확인**
- ✅ **모델 생성 성공**: `swin_base_patch4_window12_384_in22k` (86,896,009 파라미터)
- ✅ **GPU 동작 확인**: CUDA 환경에서 정상 동작
- ✅ **추론 테스트 통과**: 입력 `[1, 3, 384, 384]` → 출력 `[1, 17]`
- ✅ **결과 저장 완료**: 체계적인 로그 및 데이터 저장

### 📁 **저장된 결과 파일 구조**
```
notebooks/modular/unit_tests/swin_model_test/YYYYMMDD_HHMMSS/
├── test_summary.json          # 전체 테스트 요약 정보
├── test_results/
│   └── swin_model_test.json   # 상세 테스트 결과 데이터
├── logs/
│   └── test.log              # 실행 로그 파일
└── images/                   # (필요시) 모델 관련 시각화 이미지
```

### 📝 **저장된 데이터 내용**
- **모델 정보**: 이름, 파라미터 수, 설정값
- **입출력 정보**: 텐서 shape, 값 범위
- **실행 환경**: GPU/CPU, CUDA 버전
- **성능 지표**: 추론 시간, 메모리 사용량
- **에러 로그**: 발생한 예외 및 디버깅 정보

### 🔍 **추가 검증 방법**
- 각 셀의 출력 결과에서 ✅ 또는 ⚠️ 메시지로 성공/실패 확인
- `test_summary.json` 파일에서 전체 결과 요약 확인
- 로그 파일에서 상세한 실행 과정 및 에러 정보 확인
- GPU 메모리 사용량 및 추론 속도 모니터링

### 💡 **문제 해결 가이드**
- **모델 로드 실패**: timm 라이브러리 버전 및 모델 이름 확인
- **GPU 에러**: CUDA 환경 설정 및 메모리 가용성 점검
- **추론 실패**: 입력 텐서 형태 및 데이터타입 검증
- **저장 실패**: 디렉토리 권한 및 디스크 공간 확인